In [ ]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

In [ ]:
paths = ["./train_prep.csv","./test_prep.csv"]
target_name = "target"

In [ ]:
%%time

rd = Reader(sep = ",")

In [ ]:
%%time

df = rd.train_test_split(paths, target_name)   #reading and preprocessing (dates, ...)

In [ ]:
%%time

dft = Drift_thresholder()
df = dft.fit_transform(df)   #removing non-stable features (like ID,...)

In [ ]:
%%time

opt = Optimiser(scoring = "roc_auc", n_folds = 10)

#to evaluate models with default config
#opt = Optimiser(scoring = "roc_auc", n_folds = 10).evaluate(None, df)

In [ ]:
space = {
    
        'est__strategy':{"search":"choice",
                                  "space":["LightGBM"]},    
        'est__n_estimators':{"search":"choice",
                                  "space":[150]},    
        'est__colsample_bytree':{"search":"uniform",
                                  "space":[0.8,0.95]},
        'est__subsample':{"search":"uniform",
                                  "space":[0.8,0.95]},
        'est__max_depth':{"search":"choice",
                                  "space":[5,6,7,8,9]},
        'est__learning_rate':{"search":"choice",
                                  "space":[0.07]} 
    
        }

params = opt.optimise(space, df, 40)

In [ ]:
%%time

prd = Predictor(verbose=True)
prd.fit_predict(params, df)

# to predict with default config
#prd.fit_predict(None, df)

In [ ]:
import pandas as pd

submit = pd.read_csv("./sample_submission.csv",sep=',')
preds = pd.read_csv("save/target_predictions.csv")

submit['target'] =  preds["1.0"].values

submit.to_csv("./results-mlbox-20200531.csv", index=False)